In [1]:
# whenever raw data files have been altered, run "../scripts/xml_cleaner.py"

# Algorithm for extracting data from nested XML files (pre-cleaned)

In [2]:
import xml.etree.ElementTree as ET
import pandas as pd

# custom function for extracting data from xml and saving to pandas DataFrame
def patent_xml2df(xml_data):
    root = ET.XML(xml_data)
    all_records = []
    for patent in root:
        record = {}
        for element in patent[0]:
            if element.tag == 'publication-reference':
                for item in element[0]:
                    if item.tag == 'date':
                        record['grant_date'] = item.text
                    elif item.tag == 'doc-number':
                        record['patent_number'] = item.text
            elif element.tag == 'application-reference':
                record['application_type'] = element.get('appl-type')
                for item in element[0]:
                    if item.tag == 'date':
                        record['filing_date'] = item.text
                    elif item.tag == 'doc-number':
                        record['application_number'] = item.text
            elif element.tag == 'figures':
                for item in element:
                    record[item.tag] = item.text
            elif element.tag == 'number-of-claims':
                record['number_of_claims'] = element.text
            elif element.tag == 'invention-title':
                if element.text == None:
                    record['invention_title'] = element[0].text + element[0].tail
                else:
                    record['invention_title'] = element.text
        all_records.append(record)
    return pd.DataFrame(all_records)

# Using globbing to get names of all cleaned XML file paths &
# compiling list of DataFrames for concatenation

In [3]:
import glob

# use globbing to get all names of cleaned xml file paths
clean_files = glob.glob('../clean_data/*')


# loop over file names, extract data and concatenate into single DataFrame
xml_data_list = []
for clean_file in clean_files:
    data = open(clean_file).read()
    df = patent_xml2df(data)
    xml_data_list.append(df)
    
patents_df = pd.concat(xml_data_list, ignore_index=True)
    

# Casting Data Types for Specific Columns

In [4]:
# change the "filing_date" and "grant_date" columns to datetime objects
patents_df['filing_date'] = pd.to_datetime(patents_df['filing_date'])
patents_df['grant_date'] = pd.to_datetime(patents_df['grant_date'])

# change number-of-claims column to type int
patents_df['number_of_claims'] = patents_df['number_of_claims'].astype(int)

# Filling Null Values with 0
### (these values are omitted when value is zero)

In [5]:
patents_df['number-of-drawing-sheets'] = patents_df['number-of-drawing-sheets'].fillna('0')
patents_df['number-of-figures'] = patents_df['number-of-figures'].fillna('0')

# Displaying Resulting DataFrame

In [6]:
patents_df

,application_number,application_type,filing_date,grant_date,invention_title,number-of-drawing-sheets,number-of-figures,number_of_claims,patent_number
0,29541102,design,2015-09-30,2017-10-31,Shaped tortilla,3,6,1,D0800990
1,29539276,design,2015-09-11,2017-10-31,Utility belt,7,9,1,D0800991
2,29555649,design,2016-02-24,2017-10-31,Brassiere,7,7,1,D0800992
3,29519436,design,2015-03-05,2017-10-31,Shirt,4,4,1,D0800993
4,35500692,design,2015-09-08,2017-10-31,Footmuff for a stroller for children,7,7,1,D0800994
5,29587571,design,2016-12-14,2017-10-31,Apparel with angled stretch panel,1,1,1,D0800995
6,29601989,design,2017-04-27,2017-10-31,Shorts,5,5,1,D0800996
7,29602056,design,2017-04-27,2017-10-31,Shorts,5,5,1,D0800997
8,29602061,design,2017-04-27,2017-10-31,Shorts,6,6,1,D0800998
9,29531309,design,2015-06-24,2017-10-31,Leggings,2,3,1,D0800999


# Conclusions regarding 'Null' values:

There are 12633 patent grants that do not contain figures or drawings (and thus have no drawing sheets).

The `invention_title` elements in the raw XML files often contained nested styling tags (e.g., the italics tag &lt;i&gt; was used for genus names of plants and other living organisms).

In [7]:
patents_df.to_csv('../intermediate_csv_files/clean_raw_data.csv')